In [76]:
library(httr)
library(jsonlite)
library(utils)
library(gridExtra)

In [148]:
get_authentication_token <- function(client_id, client_secret_id){
    response = POST('https://accounts.spotify.com/api/token',
                    accept_json(),
                    authenticate(client_id, client_secret_id),
                    body = list(grant_type = 'client_credentials'),
                    encode = 'form',
                    verbose())
    authentication_token <<- content(response)$access_token
    return (authentication_token)
}

In [3]:
get_podcast_id <- function(query, market='US'){
    base_url = 'https://api.spotify.com/v1/search'
    response <- GET(base_url, 
                    query = list(q = enc2utf8(query),
                                 type = 'show',
                                 market=market),
                    add_headers(Accept = 'application/json', 
                    Authorization = paste('Bearer', authentication_token)))
    response = content(response)
    return (response$shows$items[[1]]$id)
}

In [193]:
search_for_podcast <- function(keywords, language = 'en', market='US', explicit = TRUE, limit=5){
    base_url = 'https://api.spotify.com/v1/search'
    response <- GET(base_url, 
                    query = list(q = enc2utf8(keywords),
                                 type = 'show',
                                 market=market),
                    add_headers(Accept = 'application/json', 
                    Authorization = paste('Bearer', authentication_token)))
    
    podcast_name = list()
    podcast_publisher = list()
    podcast_id = list()
    explicit_content = list()
    podcast_language = list()
    response = content(response)
    
    search_limit = response$shows$limit

    
    for (i in 1:search_limit){
        podcast_name[[i]] <- response$shows$items[[i]]$name
        podcast_publisher[[i]] <- response$shows$items[[i]]$publisher
        podcast_id[[i]] <- response$shows$items[[i]]$id
        explicit_content[[i]] <- response$shows$items[[i]]$explicit 
        podcast_language[[i]] <- response$shows$items[[i]]$language[[1]]
    }
    
    podcast_search = data.frame(unlist(podcast_name),
                                unlist(podcast_publisher),
                                unlist(explicit_content),
                                unlist(podcast_language),
                                unlist(podcast_id)
                                )
    colnames(podcast_search) <- c('Podcast Name', 'Podcast Publisher', 'Explicit','Language', 'Podcast ID')
    
    if (explicit == FALSE){
        podcast_df <- subset(podcast_search, Explicit == FALSE & Language == language)
        return(head(podcast_df, limit))
    }
    
    else{
        podcast_df <- subset(podcast_search, Language == language)
        return(head(podcast_df, limit))
        }

}

In [5]:
get_episode_information <- function(episode_id){
    base_url = 'https://api.spotify.com/v1/episodes/'
    query = paste0(episode_id,'?market=ES')
    url = paste0(base_url, query)
    response <- GET(url, 
                    add_headers(Accept = 'application/json', 
                                Authorization = paste('Bearer', authentication_token)))
    return (response)
    
}

### Test

In [152]:
client_id = Sys.getenv('CLIENT_ID')
secret_id = Sys.getenv('CLIENT_SECRET_ID')

authentication_token = get_authentication_token(client_id, secret_id)

In [7]:
episode_id = '5fE0bPigcmwSrrYxjJD4Sv'
episode = get_episode_information(episode_id)
episode = content(episode)
episode$description

[1] "Today we continue our discussion of the work of Erich Fromm. "

In [188]:
podcast_keywords = 'abroad in japan'
show_id = get_podcast_id(podcast_keywords)
show_id

[1] "1dbguTBIriCwhJXlxTITFe"

In [200]:
podcast_keywords = 'history'
response = search_for_podcast(podcast_keywords, limit=10)


In [201]:
response

,Podcast Name,Podcast Publisher,Explicit,Language,Podcast ID
,<chr>,<chr>,<lgl>,<chr>,<chr>
1,The History of Rome,Mike Duncan,FALSE,en,6wiEd40oPbQ9UK1rSpIy8I
2,Revisionist History,Pushkin Industries,FALSE,en,2LOJaYKijiwNefCvzczyib
3,American History Tellers,Wondery,FALSE,en,6ynzLDYSVQUsI0ZgMAxwWI
4,An Oral History of The Office,Propagate Content,FALSE,en,5T1519uHGVd6Te2xF5tKkR
5,Stuff You Missed in History Class,iHeartRadio,FALSE,en,4Zkj8TTa7XAZYI6aFetlec
6,History That Doesn't Suck,Prof. Greg Jackson,FALSE,en,7AVu47DIP4G9YGMJXtMgRi
7,The History Chicks,The History Chicks /Wondery,FALSE,en,7dqlIAdQaybiJrfrtCbRnW
8,The History of Byzantium,thehistoryofbyzantium@gmail.com,FALSE,en,2APexkhnuepwYbPhbrq3Mm
9,Twisted History,Barstool Sports,TRUE,en,0MAy0KsQpBaLNA0f1SFD6h
